In [23]:
mkdir .kaggle

In [27]:
ls .kaggle/

kaggle.json


In [26]:
import json

token = {"username":"andrewmaadabrima","key":"ac28103f1ac56d0bcd1ec22070c674fc"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [28]:
!chmod 600 /content/.kaggle/kaggle.json

In [30]:
!cp -R /content/.kaggle/kaggle.json /root/.kaggle/

In [3]:
import os
os.chdir('/content/drive/My Drive/')

print(os.getcwd())

/content/drive/My Drive


In [ ]:
!kaggle competitions download -c tgs-salt-identification-challenge --force

## Problem 1 Learning / estimation


In [2]:
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator

def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


In [58]:
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)


def testGenerator(test_files,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    for path in test_files:
        img = io.imread(path, as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img        

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

### study

In [62]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'./train/','images','masks',data_gen_args,save_to_dir = None)
model = unet("unet_Salt.hdf5")
model_checkpoint = ModelCheckpoint('unet_Salt.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])


Found 4000 images belonging to 1 classes.
Found 4000 images belonging to 1 classes.
Epoch 1/5
   2/2000 [..............................] - ETA: 4:54 - loss: 0.4616 - accuracy: 0.9703WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0487s vs `on_train_batch_end` time: 0.1010s). Check your callbacks.


2000/2000 [==============================] - ETA: 0s - loss: 0.5847 - accuracy: 0.7508
Epoch 00001: loss improved from inf to 0.58471, saving model to unet_Salt.hdf5
2000/2000 [==============================] - 313s 157ms/step - loss: 0.5847 - accuracy: 0.7508
Epoch 2/5
2000/2000 [==============================] - ETA: 0s - loss: 0.5782 - accuracy: 0.7504
Epoch 00002: loss improved from 0.58471 to 0.57823, saving model to unet_Salt.hdf5
2000/2000 [==============================] - 313s 156ms/step - loss: 0.5782 - accuracy: 0.7504
Epoch 3/5
2000/2000 [==============================] - ETA: 0s - loss: 0.5735 - accuracy: 0.7504
Epoch 00003: loss improved from 0.57823 to 0.57348, saving model to unet_Salt.hdf5
2000/2000 [==============================] - 314s 157ms/step - loss: 0.5735 - accuracy: 0.7504
Epoch 4/5
2000/2000 [==============================] - ETA: 0s - loss: 0.5698 - accuracy: 0.7507
Epoch 00004: loss improved from 0.57348 to 0.56983, saving model to unet_Salt.hdf5
2000/2000

In [63]:
test_files = glob.glob("./test/images/*.png")

In [ ]:
testGene = testGenerator(test_files)
results = model.predict_generator(testGene,30,verbose=1)
saveResult("./test/results",results)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

result_files = glob.glob("./test/results/*")

for i, file in enumerate(result_files):
  img = Image.open(file)
  print(np.array(img))
  plt.rcParams["figure.figsize"] = (10, 10)
  plt.subplot(3, 10, i+1)

plt.show(img)

## Problem 2 Code reading

### What is U-net
U-net is one of the FCNs and is a network for estimating image segmentation.
Published in 2015 for image segmentation in biomedical sciences.
At that time, high recognition accuracy was achieved by amplifying the data with a small amount of supervised learning by data Augumentation and using it efficiently for learning.

### FCN
A general CNN is composed of a convolutional layer and a fully connected layer, but the fully connected layer is replaced with a convolutional layer. By replacing the fully connected layer, the output of "what is the object" becomes an image of "where is the object".

### skip-connection
As the convolution process is added, the network extracts features about "what the object is", but the information about "where the object is" is lost due to the influence of Pooling. The location information may not be restored even if deconvolution is performed after the convolution process is performed. Skip-connection solves this problem. After convolution, the feature map is retained and added to the deconvolution image later.

#### Code reading

### U-net
```python
def unet(pretrained_weights = None,input_size = (256,256,1)):
    #Input layer=======================================================
    inputs = Input(input_size)
    #1st layer=======================================================
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    #MaxPooling==================================================
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #2nd layer=======================================================
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    #MaxPooling==================================================
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #n3rd layer=======================================================
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    #MaxPooling==================================================
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #4th layer=======================================================
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    #MaxPooling==================================================
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    #5th layer=======================================================
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    #UpConv======================================================
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    #SkipConnection(4th layer)========================================
    merge6 = concatenate([drop4,up6], axis = 3)
    #4th layer(right)=================================================
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    #upconv=======================================================
    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    #SkipConnection(3rd layer)========================================
    merge7 = concatenate([conv3,up7], axis = 3)
    #3rd layer(right)=================================================
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    #upconv=======================================================
    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    #SkipConnection(2nd layer)========================================
    merge8 = concatenate([conv2,up8], axis = 3)
    #2nd layer(right)=================================================
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    #upconv=======================================================
    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    #SkipConnection(1st layer)========================================
    merge9 = concatenate([conv1,up9], axis = 3)
    #1st layer(right)=================================================
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #output=======================================================
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
```